<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/Estandarizacion_marcas_modelos_Vin_dic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import os


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

In [4]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/database.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Anexo 6')
df.head()
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN


In [5]:
vin_dict = {
    "5KA": "acura", "JH4": "acura", "1HG": "acura",
    "ZAR": "alfa romeo",
    "WA1": "audi", "WAU": "audi", "WVG": "audi",
    "LGE": "baic",
    "MCA": "bajaj",
    "SCA": "bentley",
    "WBA": "bmw", "WBS": "bmw", "WBX": "bmw", "5UX": "bmw",
    "1G4": "buick", "4G1": "buick", "2G4": "buick",
    "BYD": "byd",
    "1G6": "cadillac", "2G1": "cadillac",
    "1G1": "chevrolet", "1G2": "chevrolet", "2G1": "chevrolet", "1G8": "chevrolet",
    "LC1": "changan",
    "LCA": "chery",
    "1C4": "chrysler", "2C3": "chrysler", "3C3": "chrysler",
    "VSS": "cupra",
    "KLA": "daewoo",
    "1B3": "dodge", "2B3": "dodge", "3B3": "dodge",
    "LDD": "ducati",
    "ZFA": "fiat", "3F6": "fiat",
    "1FA": "Ford", "1FB": "Ford", "1FM": "Ford", "1FT": "Ford",
    "2FA": "Ford", "2FB": "Ford", "2FM": "Ford", "2FT": "Ford",
    "3FA": "Ford", "3FB": "Ford", "3FM": "Ford", "3FT": "Ford",
    "FOT": "foton",
    "1FV": "freightliner",
    "1GK": "gmc", "2GK": "gmc",
    "LBG": "great wall",
    "7R9": "haval",
    "1HD": "harley",
    "JAC": "hino",
    "1HG": "honda", "2HG": "honda", "3HG": "honda",
    "5NP": "hyundai", "KMH": "hyundai",
    "3N1": "infiniti",
    "JA6": "isuzu",
    "SIC": "italika",
    "SAJ": "jaguar",
    "1J4": "jeep", "1C4": "jeep",
    "LFO": "jmc",
    "1XK": "kenworth",
    "5XX": "kia", "KNA": "kia", "KN1": "kia",
    "KUR": "kurazai",
    "SAL": "land rover",
    "JT2": "lexus", "JTH": "lexus",
    "5LM": "lincoln",
    "1M1": "mack",
    "ZFF": "masserati",
    "JM1": "mazda", "JM3": "mazda",
    "WDB": "mercedes-benz", "WME": "mercedes-benz", "1J4": "mercedes-benz",
    "WMW": "mini",
    "4A3": "mitsubishi", "4M5": "mitsubishi", "JA3": "mitsubishi",
    "JN1": "nissan", "1N4": "nissan", "5N1": "nissan",
    "VF3": "peugeot",
    "1XP": "peterbilt",
    "ZAP": "piaggio",
    "2G2": "pontiac",
    "WP0": "porsche", "WPO": "porsche",
    "1C6": "ram", "3C6": "ram",
    "VF1": "renault",
    "YS3": "saab",
    "YS4": "scania",
    "VSS": "seat",
    "LSG": "sinotruk",
    "JF1": "subaru",
    "2S3": "suzuki",
    "5YJ": "tesla",
    "SDA": "tiggo",
    "JT2": "toyota", "1NX": "toyota",
    "WV1": "volkswagen", "WVW": "volkswagen",
    "YV1": "volvo", "YV2": "volvo",
    "JYA": "yamaha",
    "VR1": "vento",
    "JKA": "kawazaki",
    "VBK": "ktm",
    "ZAP": "aprilia",
    "LKB": "bimota",
    "VRC": "mv agusta",
    "ME1": "royal enfield",
    "1HD": "harley-davidson",
    "5H1": "hero",
    "5HU": "husqvarna",
    "5I3": "indian",
    "LAV": "vespa",
    "LEI": "zanella",
    "ZRT": "benelli",
    "CFM": "cf moto",
    "KM2": "kymco",
    "NRT": "norton",
    "SYN": "sym",
    "SCC": "lotus",
    "VF7": "citroen",
    "LBD": "dacia",
    "WOE": "opel",
    "TM7": "skoda",
    "WME": "smart",
    "MTA": "tata",
    "MA7": "mahindra",
    "LJX": "geely",
    "PL1": "proton",
    "LAD": "lada",
    "SSY": "ssangyong",
    "MAR": "maruti",
    "1C3": "chrysler", "1G4": "saturn", "SAC": "daewoo", "3SA": "saic",
    "9X4": "zhongxing", "HA7": "haima", "1R5": "reno", "3VW": "vw", "1C6": "ram"
}



In [8]:
df["clean_serie"] = df["serie"].fillna("").astype(str).str.strip()


In [9]:
def categorize_by_vin(vin, vin_dict):
    vin_prefix = vin[:3]  # Extract the first three characters
    return vin_dict.get(vin_prefix, "Unknown")  # Get the maker or default to "Unknown"

In [11]:
# Create a new column for the categorized maker
df["maker_vin"] = df["clean_serie"].apply(lambda x: categorize_by_vin(x, vin_dict))

In [12]:
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_car_brand,clean_serie,maker_vin
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN,honda,3HGCM66563G001615,honda
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN,nissan,3N6DD25T2EK005444,Unknown
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN,volkswagen,MEX5G2603JT120368,Unknown
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN,jeep,1C4BJWEG6FL629655,jeep
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN,chevrolet,MA6CB6CD6KT029772,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN,nissan,3N1CN7AD7KK399688,infiniti
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN,mg,LSJA24U99RN010717,Unknown
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN,chevrolet,3G1SF21X97S112886,Unknown
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN,kia,LJD5AA194R0124080,Unknown


In [14]:
# Filter rows where the "maker" column is "Unknown"
unknown_maker_rows = df[df["maker_vin"] == "Unknown"]

# Print the filtered rows
print(unknown_maker_rows)


        Unnamed: 0  numcaso  tipo_vehiculo      fecha_servicio     plate  \
1              NaN   355847         Camion 2023-09-01 00:06:09   TP9498G   
2              NaN   355849           Auto 2023-09-01 00:08:38   NCZ1892   
4              NaN   355864           Auto 2023-09-01 00:46:09   NJA2747   
5              NaN   355870           Auto 2023-09-01 00:57:06    FF935B   
6              NaN   355871           Auto 2023-09-01 00:59:32  TXZ215-A   
...            ...      ...            ...                 ...       ...   
279422         NaN   745804  Equipo Ligero 2024-08-31 00:00:00   JSH4523   
279425         NaN   745810  Equipo Ligero 2024-08-31 00:00:00   JUB4510   
279426         NaN   745811  Equipo Ligero 2024-08-31 00:00:00   RTR920A   
279427         NaN   745814  Equipo Ligero 2024-08-31 00:00:00   PWEE63E   
279428         NaN   745820  Equipo pesado 2024-08-31 00:00:00   JW80882   

                            serie   model Tipo de Auto   car_brand  \
1               3

#Descarga archivo


In [16]:
file_name = 'dataframe_makers_dict.xlsx'

In [ ]:
#Descarga en Colab session

df.to_excel(file_name, index=False)

# Confirm the export (optional)
print(f"DataFrame exported successfully to {file_name}")

In [18]:
#Para descargar localmente
df.to_excel(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>